In [ ]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import base64
import json
import time

In [ ]:
def save_image(payload):
    """Save latest image as jpg file for debugging only."""
    decoded = base64.b64decode(payload['image'])  # Decode image from json
    filename = payload['camera_id'] + '.jpg' 
        with open(filename, 'wb') as f:
        f.write(decoded)
    return payload['timestamp'], payload['camera_id']

In [ ]:
# Listen to Kafka Stream

# Stream Settings
zkQuorum = 'localhost:2181'
topic = 'pycturestream'
ssc = StreamingContext(sc, 3)  # Update every 3 seconds
kvs = KafkaUtils.createStream(ssc, 
                              zkQuorum, 
                              "spark-streaming-consumer", 
                              {topic: 1})

# Transform stream payload
dstream = kvs.map(lambda x: json.loads(x[1]))  # transform json to object
images = dstream.map(lambda x: save_image(x))  # get image string & save

images.pprint(5)

# Actually start Streaming Context
ssc.start()  
ssc.awaitTermination()